In [5]:
class Polynomial:
    def __init__(self, poly_list=None):
        poly_list = poly_list or [] # Empty list handling
        self.poly_list = poly_list
        self.poly_dict = {} # Creating a dictionary to store the exponent : coefficient
        if poly_list != []: # Exception handling for empty list
            # Adding elements to the poly_dict
            for elem in poly_list:
                if elem[1] in self.poly_dict:
                    self.poly_dict[elem[1]] = self.poly_dict[elem[1]] + elem[0]
                else:
                    if elem[0] != 0:
                        self.poly_dict[elem[1]] = elem[0]
        self.rmv_empty() # Removing any empty elements
        self.key_list = sorted(self.poly_dict.keys())[::-1] # Sorting the keylist to show the higher exponents first
        #
    def iszero(self): # Zero polynomial check by checking empty dict
        return self.poly_dict == {}
        #
    def rmv_empty(self): # Removing elements with coefficient 0
        temp_poly_dict = dict(self.poly_dict) # Working on a copy while deleting from the original
        for elem in temp_poly_dict:
            if temp_poly_dict[elem] == 0:
                del self.poly_dict[elem] # Erasing entries where coefficient became 0
        #
    def __add__(self, q):
        new_poly = Polynomial([]) # Making a polynomial to hold the sum
        new_poly.poly_dict = dict(self.poly_dict) # Adding the first polynomial to the sum polynomial
        for expo in q.poly_dict: # Adding all the elements of the second elements in the sum polynomial
            if expo in new_poly.poly_dict:
                new_poly.poly_dict[expo] = new_poly.poly_dict[expo] + q.poly_dict[expo]
            else:
                new_poly.poly_dict[expo] = q.poly_dict[expo]
        new_poly.rmv_empty() # Cleaning the new sum polynomial
        new_poly.key_list = sorted(new_poly.poly_dict.keys())[::-1] # Filling the sum polynomial key_list
        return new_poly # Returning the result polynomial
        #
    def __sub__(self, q):
        new_poly = Polynomial([]) # Making a polynomial to hold the subtraction
        new_poly.poly_dict = dict(self.poly_dict) # Adding the first polynomial to the sub polynomial
        for expo in q.poly_dict: # Subtracting all the elements of the second elements in the sub polynomial
            if expo in new_poly.poly_dict:
                new_poly.poly_dict[expo] = new_poly.poly_dict[expo] - q.poly_dict[expo]
            else:
                new_poly.poly_dict[expo] = -q.poly_dict[expo]
        new_poly.rmv_empty() # Cleaning the new sub polynomial
        new_poly.key_list = sorted(new_poly.poly_dict.keys())[::-1] # Filling the sub polynomial key_list
        return new_poly # Returning the result polynomial
        #
    def __mul__(self, q):
        new_poly = Polynomial([]) # Making a polynomial to hold the multiplication
        for expo1 in q.poly_dict: # Looping over terms of the right side polynomial
            for expo2 in self.poly_dict: # Looping over terms of the left side polynomial
                if expo1 + expo2 in new_poly.poly_dict: # Adding the elements the mult poly dictionary
                    new_poly.poly_dict[expo1 + expo2] = new_poly.poly_dict[expo2 + expo1] + (self.poly_dict[expo2] * q.poly_dict[expo1]) # Combining like exponents by summing products
                else:
                    new_poly.poly_dict[expo1 + expo2] = self.poly_dict[expo2] * q.poly_dict[expo1] # First product for this exponent
        new_poly.rmv_empty() # Cleaning the new mult polynomial
        new_poly.key_list = sorted(new_poly.poly_dict.keys())[::-1] # Filling the mult polynomial key_list
        return new_poly # Returning the result polynomial
        #
    def __neg__(self):
        n = len(self.poly_list)
        new_poly_list = []
        for i in range(n):
            new_poly_list.append([-self.poly_list[i][0],self.poly_list[i][1]])
        return Polynomial(new_poly_list)
        #
    def __lt__(self, q):
        # Making copy of the dict and keylist
        self_poly_dict = self.poly_dict.copy()
        self_key_list = sorted(self.poly_dict.keys())[::-1]
        q_poly_dict = q.poly_dict.copy()
        q_key_list = sorted(q.poly_dict.keys())[::-1]

        # Handling zero polynomials
        if self_poly_dict == {}:
            self_poly_dict[0] = 0 # Represent zero with exponent 0 for comparison use
            self_key_list = sorted(self_poly_dict.keys())[::-1]
        if q_poly_dict == {}:
            q_poly_dict[0] = 0 # Same handling for the other polynomial
            q_key_list = sorted(q_poly_dict.keys())[::-1]

        # Defining Minimum length for the range check
        min_len = min(len(self_key_list), len(q_key_list)) # Compare up to the shorter length

        for i in range(min_len):
            key1 = self_key_list[i] # Current highest remaining exponent of self
            key2 = q_key_list[i] # Current highest remaining exponent of q
            if key1 < key2:
                return True # Smaller leading exponent means less
            if key2 < key1:
                return False # Larger leading exponent on self means greater
            if key1 == key2:
                pval = self_poly_dict[key1] # Coefficient for this exponent in self
                qval = q_poly_dict[key2] # Coefficient for this exponent in q
                if pval < qval:
                    return True # Same exponent so compare coefficients
                elif pval > qval:
                    return False # Coefficient is larger so not less
                else:
                    continue # Move to next term when both match
        if len(self_key_list) < len(q_key_list):
            return True
        return False # No deciding difference found so treat as not less
        #
    def __le__(self, q):
        return self.__lt__(q) or self.__eq__(q) # Lesser or equal by using or statement
        #
    def __ge__(self, q):
        return q.__le__(self) # Greater or equal by swapping the operands
        #
    def __gt__(self, q):
        return q.__lt__(self) # Greater than by reusing less than on swapped operands
        #
    def __eq__(self, q):
        for key1 in self.key_list: # Every exponent in self must be present in q
            if key1 not in q.key_list:
                return False # Missing exponent means not equal
            else:
                if q.poly_dict[key1] != self.poly_dict[key1]:
                    return False # Different coefficient means not equal
        for key2 in q.key_list: # Every exponent in q must be present in self
            if key2 not in self.key_list:
                return False # Missing exponent means not equal
            else:
                if self.poly_dict[key2] != q.poly_dict[key2]:
                    return False # Different coefficient means not equal
        return True # All exponents and coefficients match
        #
    def __ne__(self, q):
        return not self.__eq__(q) # Not equal is the logical negation of equal
        #
    def __str__(self):
        poly_print = "" # Building the printable string
        n = len(self.key_list) # Number of distinct exponents
        if n != 0:
            for i in range(n):
                key = self.key_list[i] # Current exponent
                cons = self.poly_dict[key] # Current coefficient

                if cons == 0: # might be unnecessary
                    break # Skip remaining when a zero is encountered

                if key == 0:
                    var = str(abs(cons)) # Pure constant term
                elif key == 1:
                    var = str(abs(cons)) + "x" # Linear term
                else:
                    var = str(abs(cons)) + "x^" + str(key) # General term

                if cons < 0 and i != 0:
                    var = " - " + var # Negative sign for subsequent terms
                elif cons > 0 and i != 0:
                    var = " + " + var # Positive sign for subsequent terms
                poly_print = poly_print + var # Appending this term to the output
        if poly_print == "":
            poly_print = "0" # Zero polynomial string
        return str(poly_print) # Returning the final string
        #
    def __repr__(self):
        return str(self) # Same text for interactive display

In [6]:
# ----- Basic test setup -----
p = Polynomial([(1, 0)])          # constant term 1
q = Polynomial([(1, 2)])          # x^2
r = Polynomial()                  # zero polynomial
s = Polynomial([(1, 1)])          # x

print("-------------------------")

# ----- Relational comparisons -----
print("p < q :", p < q)
print("p < r :", p < r)
print("p < s :", p < s)
print("q < r :", q < r)
print("q < s :", q < s)
print("r < s :", r < s)

print("-------------------------")

# ----- Equality and inequality -----
print("p == q :", p == q)
print("p == r :", p == r)
print("p != s :", p != s)
print("r == Polynomial() :", r == Polynomial())

print("-------------------------")

# ----- Arithmetic tests -----
print("p + s :", p + s)
print("q - s :", q - s)
print("s * s :", s * s)
print("p + q :", p + q)
print("p - q :", p - q)
print("q * r :", q * r)

print("-------------------------")

# ----- Zero and negation checks -----
print("Zero check for r:", r.iszero())
print("Negation of s:", -s)
print("s + (-s):", s + (-s))
print("Result should be zero polynomial -> iszero:", (s + (-s)).iszero())

-------------------------
p < q : True
p < r : False
p < s : True
q < r : False
q < s : False
r < s : True
-------------------------
p == q : False
p == r : False
p != s : True
r == Polynomial() : True
-------------------------
p + s : 1x + 1
q - s : 1x^2 - 1x
s * s : 1x^2
p + q : 1x^2 + 1
p - q : 1x^2 + 1
q * r : 0
-------------------------
Zero check for r: True
Negation of s: 1x
s + (-s): 0
Result should be zero polynomial -> iszero: True
